In [1]:
import os
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score


## 1- Load Credit Card Dataset

In [3]:
cc_df = pd.read_csv("outputs/credit_card_PCA.csv")

In [4]:
print(df['Class'].value_counts())

Class
0    284315
1       492
Name: count, dtype: int64


## 2- Data Splitting

In [5]:
X = df.drop('Class', axis=1).values.astype(np.float32)
y = df['Class'].values
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
X_cc = df.drop(columns=['Class'])
y_cc = df['Class']

# 5. Quick sanity check: shapes and class distribution
print("=== Credit Card Dataset ===")
print("X_cc shape:", X_cc.shape)
print("y_cc class distribution:\n", y_cc.value_counts(normalize=True))


print("\n Features shape:", X_cc.shape)
print(" Target shape:", y_cc.shape)


## 3- Cross Validation

In [8]:
# -----------------------
# 3. Cross-validation
# -----------------------
repeats = 10
folds = 5
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

results = {
    'majority': {'auc': [], 'aupr': []},
    'minority': {'auc': [], 'aupr': []}
}

## 4- Model Training

In [7]:
# --------------------------
# Generator ve Discriminator
# --------------------------

class Generator(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim // 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        recon = self.decoder(z)
        return recon, z


class Discriminator(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, latent_dim),
            nn.ReLU(),
            nn.Linear(latent_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, z):
        return self.net(z)

# --------------------------
# OCAN Trainer
# --------------------------

class OCAN:
    def __init__(self,
                 input_dim,
                 hidden_dim=64,
                 lr_g=1e-3,
                 lr_d=1e-3,
                 n_epochs=20,
                 batch_size=128, device='cpu'):
        self.device = device
        self.gen = Generator(input_dim, hidden_dim).to(device)
        self.dis = Discriminator(hidden_dim // 2).to(device)
        self.opt_g = optim.Adam(self.gen.parameters(), lr=lr_g)
        self.opt_d = optim.Adam(self.dis.parameters(), lr=lr_d)
        self.criterion_recon = nn.MSELoss()
        self.criterion_adv = nn.BCELoss()
        self.n_epochs = n_epochs
        self.batch_size = batch_size

        # create logs folder and open log file
        os.makedirs("logs", exist_ok=True)
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        self.log_file = f"logs/ocan_log_{timestamp}.txt"

    def log(self, message):
        with open(self.log_file, "a") as f:
            f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {message}\n")

    def fit(self, X):
        ds = TensorDataset(torch.tensor(X, dtype=torch.float32))
        loader = DataLoader(ds, batch_size=self.batch_size, shuffle=True)

        for epoch in range(1, self.n_epochs + 1):
            epoch_loss_d = 0.0
            epoch_loss_g = 0.0

            for (x_batch,) in loader:
                x_batch = x_batch.to(self.device)

                # --- Train Discriminator ---
                with torch.no_grad():
                    _, z_real = self.gen(x_batch)
                z_fake = torch.randn_like(z_real).to(self.device)
                d_real = self.dis(z_real)
                d_fake = self.dis(z_fake)
                loss_d = self.criterion_adv(d_real, torch.ones_like(d_real)) + \
                         self.criterion_adv(d_fake, torch.zeros_like(d_fake))
                self.opt_d.zero_grad()
                loss_d.backward()
                self.opt_d.step()

                # --- Train Generator ---
                recon, z = self.gen(x_batch)
                d_out = self.dis(z)
                loss_recon = self.criterion_recon(recon, x_batch)
                loss_g_adv = self.criterion_adv(d_out, torch.zeros_like(d_out))
                loss_g = loss_recon + 1e-2 * loss_g_adv
                self.opt_g.zero_grad()
                loss_g.backward()
                self.opt_g.step()

                epoch_loss_d += loss_d.item()
                epoch_loss_g += loss_g.item()

            avg_d = epoch_loss_d / len(loader)
            avg_g = epoch_loss_g / len(loader)

            # log and print
            self.log(f"Epoch {epoch}/{self.n_epochs} | loss_d={avg_d:.4f} | loss_g={avg_g:.4f}")
            if epoch % 10 == 0 or epoch == self.n_epochs:
                print(f"[Epoch {epoch:3d}] loss_d={avg_d:.4f}  loss_g={avg_g:.4f}")

    def score_samples(self, X):
        ds = TensorDataset(torch.tensor(X, dtype=torch.float32))
        loader = DataLoader(ds, batch_size=self.batch_size, shuffle=False)
        scores = []
        with torch.no_grad():
            for (x_batch,) in loader:
                x_batch = x_batch.to(self.device)
                recon, z = self.gen(x_batch)
                rec_err = torch.mean((recon - x_batch) ** 2, dim=1)
                d_out = self.dis(z).squeeze()
                score = rec_err + d_out  # higher score → more anomalous
                scores.append(score.cpu())
        return torch.cat(scores).numpy()

## 5- Evaluation

In [ ]:
for r in range(repeats):
    print(f"\n=== REPEAT {r+1}/{repeats} ===")
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), start=1):
        print(f"\n--- Fold {fold}/{folds} ---")
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Majority training (non-fraud = 0)
        X_train_major = X_train[y_train == 0]
        print("Training OCAN on majority (non-fraud)...")
        ocan_major = OCAN(input_dim=X.shape[1], device=device)
        ocan_major.fit(X_train_major)
        scores_major = ocan_major.score_samples(X_test)
        auc_major = roc_auc_score(y_test, scores_major)
        aupr_major = average_precision_score(y_test, scores_major)
        results['majority']['auc'].append(auc_major)
        results['majority']['aupr'].append(aupr_major)
        print(f"Majority OCAN → AUC: {auc_major:.4f}, AUPR: {aupr_major:.4f}")

        # Minority training (fraud = 1)
        X_train_min = X_train[y_train == 1]
        if len(X_train_min) > 0:
            print("Training OCAN on minority (fraud)...")
            ocan_min = OCAN(input_dim=X.shape[1], device=device)
            ocan_min.fit(X_train_min)
            scores_min = ocan_min.score_samples(X_test)
            auc_min = roc_auc_score((y_test == 1).astype(int), scores_min)
            aupr_min = average_precision_score((y_test == 1).astype(int), scores_min)
            results['minority']['auc'].append(auc_min)
            results['minority']['aupr'].append(aupr_min)
            print(f"Minority OCAN → AUC: {auc_min:.4f}, AUPR: {aupr_min:.4f}")
        else:
            print("No minority samples to train on.")

# -----------------------
# 4. Final Results
# -----------------------
print("\n===== FINAL RESULTS =====")
print(f"Majority OCAN Mean AUC:  {np.mean(results['majority']['auc']):.4f}")
print(f"Majority OCAN Mean AUPR: {np.mean(results['majority']['aupr']):.4f}")
print(f"Minority OCAN Mean AUC:  {np.mean(results['minority']['auc']):.4f}")
print(f"Minority OCAN Mean AUPR: {np.mean(results['minority']['aupr']):.4f}")

In [15]:
np.mean(results['minority']['auc'])

np.float64(0.8862528254674165)

In [16]:
np.mean(results['majority']['auc'])

np.float64(0.9061039116018754)

In [ ]:
results

## 6- ROC Curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve

# Compute ROC curves for both models
fpr_sigmoid_minor_gmm, tpr_sigmoid_minor_gmm, _ = roc_curve(y_test, scores_major)
fpr_sigmoid_major_gmm, tpr_sigmoid_major_gmm, _ = roc_curve(y_test, scores_major)

# Plot
plt.figure(figsize=(10, 7))

plt.plot(fpr_sigmoid_minor_gmm, tpr_sigmoid_minor_gmm,
         label=f"Minority OCAN (AUC = {np.mean(results['minority']['auc']):.4f})", linestyle='-')

plt.plot(fpr_sigmoid_major_gmm, tpr_sigmoid_major_gmm,
         label=f"Majority OCAN (AUC = {np.mean(results['majority']['auc']):.4f})", linestyle='--')

# Random baseline
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')

# Labels and styling
plt.title("ROC Curves - OCAN (Minority vs Majority)", fontsize=14)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()


### Save Pickled TPR and FPR values for each model

In [17]:
import pickle
import os

# Ensure the directory exists
os.makedirs("pickled_storage", exist_ok=True)

# Save minority GMM ROC variables
with open("pickled_storage/fpr_sigmoid_minor_OCAN.pkl", "wb") as f:
    pickle.dump(fpr_sigmoid_minor_gmm, f)

with open("pickled_storage/tpr_sigmoid_minor_OCAN.pkl", "wb") as f:
    pickle.dump(tpr_sigmoid_minor_gmm, f)

# Save majority GMM ROC variables
with open("pickled_storage/fpr_sigmoid_major_OCAN.pkl", "wb") as f:
    pickle.dump(fpr_sigmoid_major_gmm, f)

with open("pickled_storage/tpr_sigmoid_major_OCAN.pkl", "wb") as f:
    pickle.dump(tpr_sigmoid_major_gmm, f)
